In [ ]:
import numpy as np
from math import *
import matplotlib.cm as cm
import scipy
from scipy.signal import convolve, deconvolve
import matplotlib.pyplot as plt
import matplotlib.animation as animation  
from matplotlib.pyplot import figure
from PIL import Image, ImageDraw
from PIL import ImageFilter
from PIL import ImageColor
from pathlib import Path
import random
from matplotlib.patches import Ellipse
import glob
from skimage.util import random_noise
import time
import sklearn
from sklearn.decomposition import FastICA, PCA

# Dictionary Leaning Algorithm

In [ ]:
def S_lamb (x, lamb):
    if np.abs(x) <= lamb:
        return 0
    elif x> lamb:
        return x-lamb
    elif x<-lamb:
        return x+lamb

In [ ]:
def proximal_mapping (X, lamb):
    for i in range(X.shape[0]):
        for j in range(X.shape[1]):
            X[i][j]=S_lamb(X[i][j], lamb)

    return X

In [ ]:
# Laplacian matrix

pixel_1 = 50
pixel_2 = 50
tot_pixel = pixel_1*pixel_2 # total number of pixels used for the movie

L = np.zeros((tot_pixel, tot_pixel))

#neighbors of 0
L[0][0]=2
L[0][1]=-1
L[1][0]=-1
L[0][pixel_1] = -1
L[pixel_1][0] = -1

#neighbors of the first line of the image except last element
for i in range(1, pixel_1-1):
    L[i][i]=3
    L[i][i+1]=-1
    L[i+1][i]=-1 
    L[i][pixel_1+i]=-1
    L[pixel_1 + i][i]=-1
    
#neighbors first line last element
L[pixel_1-1][pixel_1-1]=2
L[pixel_1-1][pixel_1-2]=-1
L[pixel_1-2][pixel_1-1]=-1
L[pixel_1-1][2*pixel_1-1]=-1
L[2*pixel_1-1][pixel_1-1]=-1

#neighbors first column except last element
for i in range(1, pixel_1-1):
    L[pixel_1*(i-1)][pixel_1*i]=-1
    L[pixel_1*i][pixel_1*(i-1)]=-1
    L[pixel_1*i][pixel_1*i+1]=-1
    L[pixel_1*i+1][pixel_1*i]=-1
    L[pixel_1*i][pixel_1*(i+1)]=-1
    L[pixel_1*(i+1)][pixel_1*i]=-1
    L[pixel_1*i][pixel_1*i]=3
    
#neighbors fisrt column, last element
L[pixel_1*(pixel_1-1)][pixel_1*(pixel_1-1)]=2
L[pixel_1*(pixel_1-2)][pixel_1*(pixel_1-1)]=-1
L[pixel_1*(pixel_1-1)][pixel_1*(pixel_1-2)]=-1
L[pixel_1*(pixel_1-1)+1][pixel_1*(pixel_1-1)]=-1
L[pixel_1*(pixel_1-1)][pixel_1*(pixel_1-1)+1]=-1

#neighbors of the last line except the last element
for i in range(1, pixel_1-1):
    L[pixel_1*(pixel_1-1)+i][pixel_1*(pixel_1-1)+i-1]=-1
    L[pixel_1*(pixel_1-1)+i-1][pixel_1*(pixel_1-1)+i]=-1
    L[pixel_1*(pixel_1-1)+i][pixel_1*(pixel_1-1)+i+1]=-1
    L[pixel_1*(pixel_1-1)+i+1][pixel_1*(pixel_1-1)+i]=-1
    L[pixel_1*(pixel_1-1)+i][pixel_1*(pixel_1-2)+i]=-1
    L[pixel_1*(pixel_1-2)+i][pixel_1*(pixel_1-1)+i]=-1
    L[pixel_1*(pixel_1-1)+i][pixel_1*(pixel_1-1)+i]=3

#neighbors last line, last element
L[tot_pixel-1][tot_pixel-1]=2
L[tot_pixel-pixel_1-1][tot_pixel-1]=-1
L[tot_pixel-1][tot_pixel-pixel_1-1]=-1

#neighbors in the center
for i in range(1,pixel_1-1):
    for j in range(1, pixel_1-1):
        L[pixel_1*i + j][pixel_1*i + 1 + j]=-1
        L[pixel_1*i + j + 1][pixel_1*i + j]=-1
        L[pixel_1*i + j][pixel_1*i - 1 + j]=-1
        L[pixel_1*i + j - 1][pixel_1*i + j]=-1
        L[pixel_1*(i+1) + j][pixel_1*i + j]=-1
        L[pixel_1*i + j][pixel_1*(i+1) + j]=-1
        L[pixel_1*(i-1) + j][pixel_1*i + j]=-1
        L[pixel_1*i + j][pixel_1*(i-1) + j]=-1
        L[pixel_1*i + j][pixel_1*i + j]=4
        
#neighbors last column
for i in range(2, pixel_1):
    L[pixel_1*(i)-1][pixel_1*(i)-1]=3
    L[pixel_1*(i+1)-1][pixel_1*(i)-1]=-1
    L[pixel_1*(i)-1][pixel_1*(i+1)-1]=-1
    L[pixel_1*(i-1)-1][pixel_1*(i)-1]=-1
    L[pixel_1*(i)-1][pixel_1*(i-1)-1]=-1
    L[pixel_1*(i)-2][pixel_1*(i)-1]=-1
    L[pixel_1*(i)-1][pixel_1*(i)-2]=-1       

In [ ]:
norm_L = np.linalg.norm(L)

In [ ]:
def Loss_function_with_background(X, L, A, D, b, lamb_A, lamb_D, lamb_L, unitary):
       
    norm_X = np.linalg.norm(X-np.dot(D,A)-np.dot(b,unitary),2)**2
    
    loss = 1/2*(norm_X  + lamb_L*np.trace(np.dot(D.T,np.dot(L,D)))) + lamb_A*np.sum(np.absolute(A)) +lamb_D*np.sum(np.absolute(D))
    return loss

In [ ]:
def decomposition_cells_with_background(X, L, it, epsilon, lamb_A, lamb_D, lamb_L, nb_cells, background = True):
    
    A = np.zeros((nb_cells, X.shape[1]))
         
    #D = np.random.randint(0,2,(X.shape[0],nb_cells))*255
    #while np.linalg.norm(D) == 0:
    #    D = np.random.randint(0,2,(X.shape[0],nb_cells))*255
        
    D = np.loadtxt('D_random.txt') #because the matrix is random, 
                                  #if you want to have the same result at each run, you need to save the matrix D
        

    b = np.zeros((2500,1))
    value_b = 0
    
    unitary = np.array([np.ones(X.shape[1])])
      
    i = 0

    first_loss = Loss_function_with_background(X,L, A, D,b, lamb_A, lamb_D, lamb_L, unitary)
    
    Keep = [first_loss*1000, first_loss]
  
    while (np.linalg.norm(D)**2 + lamb_L*norm_L) != 0 and (np.linalg.norm(D)**2) != 0 and i<it and np.abs((Keep[i+1]-Keep[i])/Keep[i])>epsilon:  
        n_A = 1/(np.linalg.norm(D)**2)
        
        grad_A = np.dot(-D.T,(X-np.dot(b,unitary)))+np.dot(np.dot(D.T,D),A)
        
        tot_A = A-n_A*grad_A 
        
        A = proximal_mapping(tot_A, n_A*lamb_A)
        
        for r in range(A.shape[0]):
            for c in range(A.shape[1]):
                if A[r][c]<0:
                    A[r][c]=0

        n_D = 1/(np.linalg.norm(A)**2 + lamb_L*norm_L)
        
        grad_D = np.dot(-(X-np.dot(b,unitary)),A.T) + np.dot(np.dot(D,A),A.T) + lamb_L*np.dot(L,D)
        
        tot_D = D - n_D*grad_D
        D = proximal_mapping(tot_D, n_D*lamb_D)
        
        for r in range(D.shape[0]):
            for c in range(D.shape[1]):
                if D[r][c]<0:
                    D[r][c]=0

        if background == True :
            value_b = np.mean(X-np.dot(D,A))
            b = np.ones((2500,1))*value_b


        Keep.append(Loss_function_with_background(X, L, A, D, b, lamb_A, lamb_D, lamb_L, unitary))
        if i/10 == int(i/10):
            print(i, np.abs((Keep[i+1]-Keep[i])/Keep[i]))

        i= i +1

    print('Nombre d\'itération: ',i)
    
    print(value_b)
    
    A[np.isnan(A)] = 0
    D[np.isnan(D)] = 0
    
    return A, D, Keep, b


# Implementation

In [ ]:
A_true = np.loadtxt('A_true.txt')
D_true= np.loadtxt('D_true.txt')
X_true = np.loadtxt('X_true.txt')
X_true_noise =  np.loadtxt('X_smooth_noise_2power-3.txt') 

for i in range(10):
    plt.imshow(X_true_noise[:,i+10].reshape((50,50)), cmap='gray', vmin= np.min(X_true_noise), vmax = np.max(X_true_noise))
    
    plt.show()

In [ ]:
start_time = time.time()

d = e**(9*log(2))
l = e**(-2*log(2))
a =  e**(19*log(2))

A_test,D_test, Keep, B = decomposition_cells_with_background(X_true_noise, L, 2500, 10**(-7), a, d, l, 3, True)

print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
A_n = np.copy(A_true)
D_n = np.copy(D_true)

for b in range(D_n.shape[1]):
    D_n[:,b]=D_n[:,b]/np.max(D_n[:,b])

D_test_n = np.copy(D_test)
A_test_n = np.copy(A_test)

for c in range(D_test_n.shape[1]):
    if np.max(D_test_n[:,c]) != 0 : 
        D_test_n[:,c]=D_test_n[:,c]/np.max(D_test_n[:,c])
        
print(np.max(D_test))
        
if np.max(D_test_n) != 0:
    result = scipy.optimize.linear_sum_assignment(-np.dot(D_test_n.T,D_n))  

    for a in range(result[1].shape[0]):
        if result[1][a] != a:
            D_test[:,result[1][a]]=D_test_n[:,a]*255
            A_test[result[1][a],:]=A_test_n[a,:]

if np.max(D_test) != 0:
    D_test = D_test/np.max(D_test)*255

fig, ax = plt.subplots(3,2, figsize=(15, 10))
plt.gcf().subplots_adjust(left = 0.125, bottom = 0.1, right = 0.9, top = 0.9, wspace = 0.2, hspace = 0.7) 

ax[0][0].imshow(D_true[:,0].reshape((50,50)), cmap='gray', vmin=0, vmax=255)
ax[0][0].set_title('True position of the 1rst cell')
ax[0][1].imshow(D_test[:,0].reshape((50,50)), cmap='gray') #, vmin=0, vmax=255)
ax[0][1].set_title('Suppose position of the 1rst cell')

ax[1][0].imshow(D_true[:,1].reshape((50,50)), cmap='gray', vmin=0, vmax=255)
ax[1][0].set_title('True position of the 2nd cell')
ax[1][1].imshow(D_test[:,1].reshape((50,50)), cmap='gray') #, vmin=0, vmax=255)
ax[1][1].set_title('Suppose position of the 2nd cell')

ax[2][0].imshow(D_true[:,2].reshape((50,50)), cmap='gray', vmin=0, vmax=255)
ax[2][0].set_title('True position of the 3rd cell')
ax[2][1].imshow(D_test[:,2].reshape((50,50)), cmap='gray') #, vmin=0, vmax=255)
ax[2][1].set_title('Suppose position of the 3rd cell')


A_test_b = np.copy(A_test)
for r in range(3):
    if np.min(A_test_b[r,:])>0:
        A_test_b[r,:] = A_test_b[r,:]-np.min(A_test_b[r,:])
    if np.max(A_test_b[r,:]) != 0:
        A_test_b[r,:] = A_test_b[r,:]/np.max(A_test_b[r,:])

    

fig, ax = plt.subplots(3,1, figsize=(15, 10))
plt.gcf().subplots_adjust(left = 0.125, bottom = 0.1, right = 0.9, top = 0.9, wspace = 0.2, hspace = 0.8) 

ax[0].plot(np.arange(0,300,1), A_true[0,:].T, 'g')
ax[0].plot(np.arange(0,300,1), A_test_b[0,:].T, 'r--')
ax[0].set_title('Superposition of the signals for the 1rs cell', fontsize = 20)
ax[0].set_xlabel('Sample', fontsize = 15)

ax[1].plot(np.arange(0,300,1), A_true[1,:].T, 'g')
ax[1].plot(np.arange(0,300,1), A_test_b[1,:].T, 'r--')
ax[1].set_title('Superposition of the signals for the 2nd cell', fontsize=20)
ax[1].set_xlabel('Sample', fontsize = 15)

ax[2].plot(np.arange(0,300,1), A_true[2,:].T, 'g')
ax[2].plot(np.arange(0,300,1), A_test_b[2,:].T, 'r--')
ax[2].set_title('Superposition of the signals for the 3rd cell', fontsize=20)
ax[2].set_xlabel('Sample', fontsize = 15)



In [ ]:
print('norm D', np.linalg.norm(D_true-D_test))
print('norm D relatif', np.linalg.norm(D_true-D_test)/np.linalg.norm(D_true))

print('norm A', np.linalg.norm(A_true-A_test_b))
print('norm A relatif', np.linalg.norm(A_true-A_test_b)/np.linalg.norm(A_true))

min_A = 1
min_D = 1

mean_A = 0
mean_D = 0
for i in range(3):
    j = abs(np.corrcoef(A_true[i,:], A_test_b[i,:])[0][1])
    if np.isnan(j):
        j = 0    
    print('corr A' + str(i), j)
    
    if j<min_A:
        min_A = j
    mean_A = mean_A + j
    
    
    j = abs(np.corrcoef(D_true[:,i], D_test[:,i])[0][1])
    if np.isnan(j):
        j = 0  
    print('corr D' + str(i), j)
    
    if j < min_D:
        min_D = j
    mean_D = mean_D + j
    
    
print('min corr A ', min_A)
print('min corr D', min_D)
print('mean A corr', mean_A/3)
print('mean D corr', mean_D/3)

# Hyper-parameter selection

In [ ]:
g = 10
h = 15
k = 10


Diff_D_tot_h = np.zeros((g,h,k))
Diff_A_h = np.zeros((g,h,k))
Error_tot_h = np.zeros((g,h,k))
Diff_D_tot_nh= np.zeros((g,h,k))
Error_tot_nh = np.zeros((g,h,k))
    
Corr_A_min = np.zeros((g,h,k))
Corr_D_min = np.zeros((g,h,k))
Mean_corr_A = np.zeros((g,h,k))
Mean_corr_D = np.zeros((g,h,k))
    
Corr_X_min = np.zeros((g,h,k))
Mean_corr_X = np.zeros((g,h,k))
    
Corr_X_min_noise = np.zeros((g,h,k))
Mean_corr_X_noise = np.zeros((g,h,k))
    

    
l_num = 0
for l in np.arange(-5,5,1):
    d_num = 0
    print('l', str(l))
    for d in np.arange(0,15,1):
        print('d', str(d))
        a_num = 0
        for a_ in np.arange(15,25,1):
            print('l', str(l), ' d', str(d), 'a', str(a_))

            start_time = time.time()

            A_test,D_test,Keep, B= decomposition_cells_with_background(X_true_noise,L, 2500, 10**(-7),e**(a_*log(2)), e**(d*log(2)), e**(l*log(2)), 3, True)


            for e in range(3):
                D_test[:,e] = D_test[:,e] - np.min(D_test[:,e]) 
                if np.max(D_test[:,e]) != 0 :
                    D_test[:,e] = D_test[:,e]/np.max(D_test[:,e])*255

                if np.min(A_test[e,:])>0:
                    A_test[e,:] = A_test[e,:] - np.min(A_test[e,:]) 
                if np.max(A_test[e,:]) != 0:
                    A_test[e,:] = A_test[e,:]/np.max(A_test[e,:])

            A_n = np.copy(A_true)
            D_n = np.copy(D_true)

            for b in range(D_n.shape[1]):
                D_n[:,b]=D_n[:,b]/np.max(D_n[:,b])

            D_test_n = np.copy(D_test)
            for c in range(D_test_n.shape[1]):
                if np.max(D_test_n[:,c]) != 0:
                    D_test_n[:,c]=D_test_n[:,c]/np.max(D_test_n[:,c])

            result = scipy.optimize.linear_sum_assignment(-np.dot(D_n.T,D_test_n))  

            for change in range(result[1].shape[0]):
                if result[1][change] != change:
                    D_true[:,result[1][change]]=D_n[:,change]*255
                    A_true[result[1][change],:]=A_n[change,:]

            X_new = np.dot(D_true, A_true)
            X_new = X_new/np.max(X_new)*255
            
            X_obt = np.dot(D_test,A_test)
            X_obt = X_obt/np.max(X_obt)

            Diff_D_tot_h[l_num][d_num][a_num] = (np.linalg.norm(D_true[:,0]-D_test[:,0],2) + np.linalg.norm(D_true[:,1]-D_test[:,1],2) + np.linalg.norm(D_true[:,2]-D_test[:,2],2))
            Error_tot_h[l_num][d_num][a_num]= (np.linalg.norm(X_new-X_obt,2))  
            Diff_A_h[l_num][d_num][a_num] = (np.linalg.norm(A_true[0,:]-A_test[0,:],2) + np.linalg.norm(A_true[1,:]-A_test[1,:],2) + np.linalg.norm(A_true[2,:]-A_test[2,:],2))


            min_A = 1
            min_D = 1

            mean_A = 0
            mean_D = 0
            for i in range(3):
                j = np.abs(np.corrcoef(A_true[i,:], A_test[i,:])[0][1])
                if np.isnan(j):
                    j = 0

                if j<min_A:
                    min_A = j
                mean_A = mean_A + j
                j = np.abs(np.corrcoef(D_true[:,i], D_test[:,i])[0][1])
                if np.isnan(j):
                    j = 0
                if j < min_D:
                    min_D = j
                mean_D = mean_D + j


            Corr_A_min[l_num][d_num][a_num] = (min_A)
            Corr_D_min[l_num][d_num][a_num] = (min_D)
            Mean_corr_A[l_num][d_num][a_num] = (mean_A/3)
            Mean_corr_D[l_num][d_num][a_num] = (mean_D/3)


            print("--- %s seconds ---" % (time.time() - start_time))
            a_num = a_num +1
        d_num = d_num +1

    l_num = l_num +1
       
print('end')